In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/video-test/ball_tiled_output.mp4


In [2]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [3]:
# Import necessary libraries
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import easyocr
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Initialize YOLO model
model = YOLO('yolov8x.pt')  # Single model for both players and ball

100%|██████████| 131M/131M [00:00<00:00, 283MB/s] 


In [5]:
# COCO class IDs
PERSON_CLASS_ID = 0
BALL_CLASS_ID = 32

In [6]:
def get_team_color(player_roi):
    """Get dominant jersey color (excluding field green)"""
    if player_roi.size == 0:
        return None
    
    hsv = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)
    
    # Mask out green field
    lower_green = np.array([35, 50, 50])
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    mask = cv2.bitwise_not(mask)
    
    # Get dominant non-green color
    pixels = player_roi[mask > 0]
    if len(pixels) == 0:
        return None
        
    pixels = np.float32(pixels)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 0.1)
    _, labels, centers = cv2.kmeans(pixels, 2, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    return centers[np.argmax(np.bincount(labels.flatten()))]

In [7]:
def assign_teams(frame, player_boxes):
    """Assign players to teams based on jersey colors"""
    colors = []
    valid_indices = []
    
    for i, box in enumerate(player_boxes):
        x1, y1, x2, y2 = map(int, box)
        roi = frame[y1:y2, x1:x2]
        color = get_team_color(roi)
        if color is not None:
            colors.append(color)
            valid_indices.append(i)
    
    # Default all to team 0
    assignments = [0] * len(player_boxes)
    
    if len(colors) >= 2:  # Need at least 2 colors to cluster
        colors_np = np.array(colors, dtype=np.float32)
        _, labels, _ = cv2.kmeans(colors_np, 2, None, 
                                 (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0), 
                                 10, cv2.KMEANS_RANDOM_CENTERS)
        
        # Map back to original boxes using indices
        for idx, label in zip(valid_indices, labels.flatten()):
            assignments[idx] = int(label)
    
    return assignments


In [8]:
def process_video(input_video_path, output_video_path='output_video.mp4', output_csv='results.csv'):
    # Open video file
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error opening video {input_video_path}")
        return None
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Create video writer for output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    results = []
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_count += 1
        if frame_count % 5 != 0:  # Skip frames for faster processing
            continue
            
        # Make a copy of the frame for visualization
        vis_frame = frame.copy()
        
        # Run detection
        detections = model(frame, verbose=False)[0]
        
        # Separate players and balls
        player_boxes = []
        ball_boxes = []
        
        for box, cls in zip(detections.boxes.xyxy.cpu().numpy(), 
                            detections.boxes.cls.cpu().numpy()):
            if cls == PERSON_CLASS_ID:
                player_boxes.append(box)
            elif cls == BALL_CLASS_ID:
                ball_boxes.append(box)
        
        # Team assignment
        teams = assign_teams(frame, player_boxes)
        
        # Process players
        for i, box in enumerate(player_boxes):
            x1, y1, x2, y2 = map(int, box)
            
            # Determine color based on team
            team = teams[i] if i < len(teams) else 0
            color = (255, 0, 0) if team == 0 else (0, 0, 255)  # Red or Blue
            
            # Draw bounding box and label
            cv2.rectangle(vis_frame, (x1, y1), (x2, y2), color, 2)
            label = f"Player T{team}"
            cv2.putText(vis_frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # Store results
            results.append({
                'frame': frame_count,
                'type': 'player',
                'team': team,
                'number': None,
                'bbox': [x1, y1, x2, y2]
            })
        
        # Process balls
        for box in ball_boxes:
            x1, y1, x2, y2 = map(int, box)
            
            # Draw ball bounding box (yellow)
            cv2.rectangle(vis_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.putText(vis_frame, "Ball", (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
            
            # Store results
            results.append({
                'frame': frame_count,
                'type': 'ball',
                'team': None,
                'number': None,
                'bbox': [x1, y1, x2, y2]
            })
        
        # Write frame to output video
        out.write(vis_frame)
        
        # Print progress
        print(f"Processed frame {frame_count}", end='\r')
    
    # Release resources
    cap.release()
    out.release()
    
    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\nResults saved to {output_csv}")
    print(f"Output video saved to {output_video_path}")
    
    return df

In [9]:
# Run if script is executed directly
if __name__ == "__main__":
    input_video = "/kaggle/input/video-test/ball_tiled_output.mp4"
    if os.path.exists(input_video):
        results = process_video(
            input_video_path=input_video,
            output_video_path="processed_output.mp4",
            output_csv="tracking_results.csv"
        )
    else:
        print(f"Input video file {input_video} not found!")

Processed frame 740
Results saved to tracking_results.csv
Output video saved to processed_output.mp4


In [15]:
!pip install -q moviepy

from moviepy.editor import VideoFileClip

# Load and re-encode the video
clip = VideoFileClip("/kaggle/working/processed_output.mp4")
clip.write_videofile("/kaggle/working/processed_output_fixed.mp4", codec='libx264')


error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu

Moviepy - Building video /kaggle/working/processed_output_fixed.mp4.
Moviepy - Writing video /kaggle/working/processed_output_fixed.mp4



Moviepy - Done !
Moviepy - video ready /kaggle/working/processed_output_fixed.mp4


In [16]:
from IPython.display import HTML

video_path = '/kaggle/working/processed_output_fixed.mp4'

HTML(f"""
<video width="640" height="480" controls autoplay loop muted>
  <source src="{video_path}" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")
